### Imports

In [1]:
import pandas as pd
import xml.etree.ElementTree as ET

### Analyse and identify traffic/emission heavy junction

In [2]:
def calculate_junction_emissions(xml_file):
    
    '''
    Function to calculate the total emissions for each pollutant at each junction.
    '''
    
    tree = ET.parse(xml_file)
    root = tree.getroot()

    junction_emissions = {}

    for timestep in root.findall('timestep'):
        for vehicle in timestep.findall('vehicle'):
            junction = vehicle.get('lane') # Extract junction ID from lane ID
            emissions = {
                'CO2': float(vehicle.get('CO2')),
                'CO': float(vehicle.get('CO')),
                'HC': float(vehicle.get('HC')),
                'NOx': float(vehicle.get('NOx')),
                'PMx': float(vehicle.get('PMx')),
                'fuel': float(vehicle.get('fuel')),
                'noise': float(vehicle.get('noise'))
            }

            if junction in junction_emissions:
                for pollutant in emissions:
                    junction_emissions[junction][pollutant] += emissions[pollutant]
            else:
                junction_emissions[junction] = emissions

    return junction_emissions


In [3]:
#junction_emissions = calculate_junction_emissions('../src/data/actuated_output/2023-07-20-22-03-05emission_info_actuated_TL.xml')

# get emission info from new demand model
junction_emissions = calculate_junction_emissions('../src/data/actuated_output/2023-08-14-12-19-51newModelDemand_emission_actuated_TL.xml')

In [4]:
def find_emission_heavy_junctions(junction_emissions, pollutant='CO2', num_junctions=5):
    '''
    Function to find the junctions with the highest emissions of a given pollutant.
    '''
    sorted_junctions = sorted(junction_emissions.items(), key=lambda x: x[1][pollutant], reverse=True)
    return sorted_junctions[:num_junctions]

In [5]:
junction_emissions

{'-35014373#0_1': {'CO2': 8494329.909999995,
  'CO': 44339.239999999976,
  'HC': 228.86999999999986,
  'NOx': 47276.63999999999,
  'PMx': 1005.6899999999957,
  'fuel': 2685758.609999997,
  'noise': 126522.62000000008},
 '187057791#1_1': {'CO2': 5624783.359999991,
  'CO': 56427.80999999981,
  'HC': 277.4000000000005,
  'NOx': 31586.760000000126,
  'PMx': 686.870000000001,
  'fuel': 1778682.649999997,
  'noise': 80941.34000000021},
 '85287424_0': {'CO2': 55495957.09999577,
  'CO': 1517477.7000001136,
  'HC': 7319.900000001185,
  'NOx': 291774.4000000672,
  'PMx': 7511.459999999335,
  'fuel': 17583486.980001632,
  'noise': 904438.6399998215},
 '-114928357_0': {'CO2': 27133437.569999166,
  'CO': 683166.410000035,
  'HC': 3257.7999999998665,
  'NOx': 157723.52000000165,
  'PMx': 3924.490000000303,
  'fuel': 8589358.949999783,
  'noise': 412075.5799999949},
 '24573665_1': {'CO2': 3375990.4900000016,
  'CO': 33782.54000000001,
  'HC': 168.71,
  'NOx': 19067.969999999998,
  'PMx': 424.78999999

In [6]:
sorted_junctions = find_emission_heavy_junctions(junction_emissions, pollutant='CO2', num_junctions=5)

In [7]:
sorted_junctions

[('96747828#0_1',
  {'CO2': 397368286.20050836,
   'CO': 12006396.499986868,
   'HC': 56754.09999995939,
   'NOx': 2392824.9299984607,
   'PMx': 64022.43999996106,
   'fuel': 125840054.91997017,
   'noise': 6053109.560007019}),
 ('27247877_0',
  {'CO2': 378750181.74024063,
   'CO': 9812217.829999961,
   'HC': 46596.57999998699,
   'NOx': 2206913.4199983897,
   'PMx': 56024.09999997148,
   'fuel': 119914311.49998602,
   'noise': 6143053.600004185}),
 ('268931658_3',
  {'CO2': 247623896.14997151,
   'CO': 8738172.150004491,
   'HC': 41880.0499999969,
   'NOx': 1318951.80999856,
   'PMx': 36057.409999977106,
   'fuel': 78498323.77997923,
   'noise': 3962638.889997454}),
 ('280873251#0_0',
  {'CO2': 241370461.83000374,
   'CO': 8124090.160004755,
   'HC': 56407.71999995424,
   'NOx': 1318148.119998663,
   'PMx': 37290.86999997846,
   'fuel': 76527172.33997029,
   'noise': 3963236.2699976577}),
 ('183629216#0_3',
  {'CO2': 223515201.81999132,
   'CO': 6748114.4000030365,
   'HC': 32043.0400

-> get junctions of respective lanes and choose for RL agent

In [6]:
# read model output
df = pd.read_csv('../src/data/model_outputs/ppo_new_local_test_conn0_ep3.csv')
df

,step,system_total_stopped,system_total_waiting_time,system_mean_waiting_time,system_mean_speed,system_total_CO2,system_total_noise_emission,tls_160_stopped,tls_160_accumulated_waiting_time,tls_160_average_speed,...,tls_161_accumulated_waiting_time,tls_161_average_speed,tls_159_stopped,tls_159_accumulated_waiting_time,tls_159_average_speed,agents_total_stopped,agents_total_accumulated_waiting_time,TimeLimit.truncated,episode,terminal_observation
0,10000.0,0,0.0,0.000000,0.000000,0.000000e+00,0.000000,0,0.0,1.000000,...,0.0,1.000000,0,0.0,1.0,0,0.0,NaN,NaN,NaN
1,10005.0,0,0.0,0.000000,0.000000,0.000000e+00,0.000000,0,0.0,1.000000,...,0.0,1.000000,0,0.0,1.0,0,0.0,False,NaN,NaN
2,10010.0,0,0.0,0.000000,0.000000,0.000000e+00,0.000000,0,0.0,1.000000,...,0.0,1.000000,0,0.0,1.0,0,0.0,False,NaN,NaN
3,10015.0,0,0.0,0.000000,0.000000,0.000000e+00,0.000000,0,0.0,1.000000,...,0.0,1.000000,0,0.0,1.0,0,0.0,False,NaN,NaN
4,10020.0,0,0.0,0.000000,0.000000,0.000000e+00,0.000000,0,0.0,1.000000,...,0.0,1.000000,0,0.0,1.0,0,0.0,False,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6636,43180.0,14718,3145084.0,205.628245,0.105681,4.187031e+07,723344.186446,49,20514.0,0.000000,...,12578.0,0.000000,67,19712.0,0.0,140,52804.0,False,NaN,NaN
6637,43185.0,14693,3150837.0,205.963982,0.115104,4.188013e+07,723506.182900,49,20524.0,0.000000,...,12593.0,0.000000,67,19729.0,0.0,140,52846.0,False,NaN,NaN
6638,43190.0,14642,3152299.0,205.965305,0.123057,4.186921e+07,723719.062057,49,20537.0,0.000000,...,12608.0,0.000000,67,19752.0,0.0,140,52897.0,False,NaN,NaN
6639,43195.0,14637,3148239.0,205.632854,0.121905,4.188205e+07,723839.705169,49,20571.0,0.000000,...,11626.0,0.012424,67,19771.0,0.0,137,51968.0,False,NaN,NaN


In [5]:
df.describe()

,step,system_total_stopped,system_total_waiting_time,system_mean_waiting_time,system_mean_speed,system_total_CO2,system_total_noise_emission,tls_160_stopped,tls_160_accumulated_waiting_time,tls_160_average_speed,tls_161_stopped,tls_161_accumulated_waiting_time,tls_161_average_speed,tls_159_stopped,tls_159_accumulated_waiting_time,tls_159_average_speed,agents_total_stopped,agents_total_accumulated_waiting_time
count,6641.000000,6641.000000,6.641000e+03,6641.000000,6641.000000,6.641000e+03,6641.000000,6641.000000,6641.000000,6641.000000,6641.00000,6641.000000,6641.000000,6641.000000,6641.000000,6641.000000,6641.000000,6641.000000
mean,26600.000000,3654.438789,6.357434e+05,83.531352,1.586697,1.259468e+07,201706.280430,23.338353,9126.659238,0.306887,9.37675,7148.110375,0.333801,32.151634,8748.576419,0.231106,64.866737,25023.346032
std,9586.179505,4316.743461,8.983318e+05,68.686517,1.853424,1.258312e+07,214733.667029,18.734531,8404.466223,0.445700,8.75790,6402.811081,0.444498,25.417858,7157.378852,0.399176,51.409584,20967.194733
min,10000.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,18300.000000,3.000000,2.180000e+02,22.292308,0.205616,1.057848e+04,134.221341,0.000000,0.000000,0.000003,0.00000,0.000000,0.000000,1.000000,294.000000,0.000000,1.000000,320.000000
50%,26600.000000,1799.000000,1.698540e+05,69.107349,0.910014,8.818753e+06,126479.614213,28.000000,10055.000000,0.019172,9.00000,7364.000000,0.057258,39.000000,10359.000000,0.010155,77.000000,28840.000000
75%,34900.000000,6455.000000,9.883510e+05,142.146944,2.468630,2.156492e+07,347988.733153,40.000000,15530.000000,1.000000,13.00000,10523.000000,1.000000,56.000000,14647.000000,0.160928,107.000000,40608.000000
max,43200.000000,14274.000000,3.312842e+06,298.000000,13.890000,4.094029e+07,703643.250516,57.000000,27385.000000,1.000000,32.00000,26085.000000,1.000000,71.000000,26502.000000,1.000000,153.000000,67187.000000
